In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from math import atan2, pi, sqrt, cos, sin, floor
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
from keras_flops import get_flops
import pickle
import csv
import random
import itertools
import math
import time
from dataset0 .data_utils_0 import *
from traj_utils import *

## Import Dataset

In [ ]:
window_size = 100
stride = 20

f = 'dataset0/'
X_train,Y_Pos_train, Physics_Vec_train, x_vel_train, y_vel_train, x0_list_train, y0_list_train, size_of_each_train = import_agrobot_dataset_p1(dataset_folder=f, type_flag=1, window_size=window_size, stride=stride)
P = np.repeat(Physics_Vec_train,window_size).reshape((Physics_Vec_train.shape[0],window_size,1))
X_train = np.concatenate((X_train,P),axis=2)

X_test,Y_Pos_test, Physics_Vec_test, x_vel_test, y_vel_test, x0_list_test, y0_list_test, size_of_each_test= import_agrobot_dataset_p1(type_flag = 2, dataset_folder=f,window_size=window_size, stride=stride)
P_test = np.repeat(Physics_Vec_test,window_size).reshape((Physics_Vec_test.shape[0],window_size,1))
X_test = np.concatenate((X_test,P_test),axis=2)

## Ablation Study Knobs


In [ ]:
mag = 0
physics = 0
velocity = 0
model_name = 'Agrobot_dset0_noMPV.hdf5'

In [ ]:
def abs_heading(cur_x, cur_y, prev_x, prev_y):
        dely = (cur_y - prev_y)
        delx = (cur_x - prev_x)
        delh= atan2(delx,dely)*57.2958
        return delh

In [ ]:
disp_train = np.sqrt((x_vel_train**2) + (y_vel_train**2)) 
disp_test = np.sqrt((x_vel_test**2) + (y_vel_test**2)) 

heading_train = np.zeros((Y_Pos_train.shape[0]))
prev = 0
for i in range(Y_Pos_train.shape[0]):
    theta = abs_heading(Y_Pos_train[i,-1,0],Y_Pos_train[i,-1,1],Y_Pos_train[i,0,0],Y_Pos_train[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_train[i] = theta - prev
    if(heading_train[i]>100 or heading_train[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_train[i] = theta2 - prev2 
    prev = theta
    
heading_test = np.zeros((Y_Pos_test.shape[0]))
prev = 0
for i in range(Y_Pos_test.shape[0]):
    theta = abs_heading(Y_Pos_test[i,-1,0],Y_Pos_test[i,-1,1],Y_Pos_test[i,0,0],Y_Pos_test[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_test[i] = theta - prev
    if(heading_test[i]>100 or heading_test[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_test[i] = theta2 - prev2 
    prev = theta

In [ ]:
if(mag==0 and physics==1):
    X_train = X_train[:,:,[0,1,2,3,4,5,9]]
    X_test = X_test[:,:,[0,1,2,3,4,5,9]]

elif(mag==1 and physics==0):
    X_train = X_train[:,:,0:9]
    X_test = X_test[:,:,0:9]

elif(mag==0 and physics==0):
    X_train = X_train[:,:,0:6]
    X_test = X_test[:,:,0:6]

else:
    print('Physics and Magnetometer enabled.')

## Model Training

In [ ]:
nb_filters = 32
kernel_size = 5
dilations = [1,2,4,8,16,32,64,128]
dropout_rate = 0.0
use_skip_connections = True

batch_size, timesteps, input_dim = 256, window_size, X_train.shape[2]
i = Input(shape=(timesteps, input_dim))

m = TCN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,dropout_rate=dropout_rate,
            use_skip_connections=use_skip_connections)(i)  

m = tf.reshape(m, [-1, nb_filters, 1])
m = MaxPooling1D(pool_size=(2))(m)
m = Flatten()(m)
m = Dense(32, activation='linear', name='pre')(m)
output1 = Dense(1, activation='linear', name='velx')(m)
output2 = Dense(1, activation='linear', name='vely')(m)
model = Model(inputs=[i], outputs=[output1, output2])
opt = tf.keras.optimizers.Adam()
model.compile(loss={'velx': 'mse','vely':'mse'},optimizer=opt)  
model.summary()

In [ ]:

checkpoint = ModelCheckpoint(model_name, monitor='loss', verbose=1, save_best_only=True)
if(velocity==1):
    model.fit(x=X_train, y=[x_vel_train, y_vel_train],
              epochs=3000, shuffle=True,callbacks=[checkpoint],batch_size=batch_size)     
else:
    model.fit(x=X_train, y=[disp_train, heading_train*0.0174533],
              epochs=3000, shuffle=True,callbacks=[checkpoint],batch_size=batch_size) 

## Evaluation

In [ ]:
def agrobot_head_disp_pos_generator(net_inp_mat, size_of_each, 
                   x0_list, y0_list, window_size, stride,file_idx,my_model):
    
    if (file_idx == 0):
        cur_inp = net_inp_mat[0:size_of_each[0],:,:]
    elif (file_idx == 1):
        cur_inp = net_inp_mat[np.sum(size_of_each[0]):np.sum(size_of_each[0:file_idx+1]),:,:]

    else:
        cur_inp = net_inp_mat[np.sum(size_of_each[0:file_idx]):np.sum(size_of_each[0:file_idx+1]),:,:]    
    
    y_pred = my_model.predict(cur_inp)
    
    pointx = []
    pointy = []
    Lx =  x0_list[file_idx]
    Ly = y0_list[file_idx]
    for i in range(len(cur_inp)):
        Lx = Lx + (y_pred[0][i]/(((window_size-stride)/stride)))*cos(y_pred[1][i])
        Ly = Ly + (y_pred[0][i]/(((window_size-stride)/stride)))*sin(y_pred[1][i])
        pointx.append(Lx)
        pointy.append(Ly)
    Pvx = pointx
    Pvy = pointy   
    
    return Pvx, Pvy

In [ ]:
def agrobot_head_disp_GT_pos_generator(disp, head, size_of_each, 
                   x0_list, y0_list, window_size, stride,file_idx):  
    if (file_idx == 0):
        disp_sel = disp[0:size_of_each[0]]
        head_sel = head[0:size_of_each[0]]
    elif (file_idx == 1):
        disp_sel = disp[np.sum(size_of_each[0]):np.sum(size_of_each[0:file_idx+1])]
        head_sel = head[np.sum(size_of_each[0]):np.sum(size_of_each[0:file_idx+1])]

    else:
        disp_sel = disp[np.sum(size_of_each[0:file_idx]):np.sum(size_of_each[0:file_idx+1])]
        head_sel = head[np.sum(size_of_each[0:file_idx]):np.sum(size_of_each[0:file_idx+1])]

    head_sel = head_sel*0.0174533
    pointx = []
    pointy = []
    Lx =  x0_list[file_idx]
    Ly = y0_list[file_idx]
    for i in range(len(disp_sel)):
        Lx = Lx + (disp_sel[i]/(((window_size-stride)/stride)))*cos(head_sel[i])
        Ly = Ly + (disp_sel[i]/(((window_size-stride)/stride)))*sin(head_sel[i])  
        pointx.append(Lx)
        pointy.append(Ly)   
    Gvx = pointx
    Gvy = pointy
    
    return Gvx, Gvy

In [ ]:
#model_name = 'Agrobot_First_TCN3.hdf5'
model = load_model(model_name,custom_objects={'TCN':TCN})

Unseen Trajectories

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_test)):
    if(velocity==1):
        Pvx, Pvy = model_pos_generator(X_test, size_of_each_test, 
                   x0_list_test, y0_list_test, window_size, stride,i,model) 
        Gvx, Gvy = GT_pos_generator(x_vel_test,y_vel_test,size_of_each_test,
                                x0_list_test, y0_list_test, window_size, stride,i)
    else:
        Pvx, Pvy = agrobot_head_disp_pos_generator(X_test, size_of_each_test, 
                   x0_list_test, y0_list_test, window_size, stride,i,model) 
        Gvx, Gvy = agrobot_head_disp_GT_pos_generator(disp_test,heading_test,size_of_each_test,
                                x0_list_test, y0_list_test, window_size, stride,i)
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))

Seen Trajectories

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_train)):
    if(velocity==1):
        Pvx, Pvy = model_pos_generator(X_train, size_of_each_train, 
                   x0_list_train, y0_list_train, window_size, stride,i,model)
        Gvx, Gvy = GT_pos_generator(x_vel_train,y_vel_train,size_of_each_train,
                                x0_list_train, y0_list_train, window_size, stride,i)
    else:
        Pvx, Pvy = agrobot_head_disp_pos_generator(X_train, size_of_each_train, 
                   x0_list_train, y0_list_train, window_size, stride,i,model)   

        Gvx, Gvy = agrobot_head_disp_GT_pos_generator(disp_train,heading_train,size_of_each_train,
                                x0_list_train, y0_list_train, window_size, stride,i)    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))